In [5]:
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, ConcatDataset
from PIL import Image
import torch
import timm
from transformers import BeitForImageClassification, DeiTForImageClassification, Swinv2ForImageClassification, ConvNextForImageClassification, ConvNextV2ForImageClassification, Dinov2ForImageClassification
from torch.optim import Adamax, RMSprop, SGD
from torch.optim.lr_scheduler import StepLR
from tqdm import tqdm 
import time
import timm
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np


In [6]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
])

data_dir = os.path.join('datasets', 'Boya3')
print(data_dir)

fold_dirs = [fold for fold in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, fold)) and "fold" in fold]
fold_results = {}
print(fold_dirs)

os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
print(os.environ["PYTORCH_CUDA_ALLOC_CONF"])

datasets/Boya3
['fold3', 'fold1', 'fold4', 'fold5', 'fold2']
expandable_segments:True


In [7]:
def train_model(fold_idx):
    model = Dinov2ForImageClassification.from_pretrained(
        "facebook/dinov2-small",
        num_labels=18,
        ignore_mismatched_sizes=True
    )

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    print(device)
    print(model)

    train_path = os.path.join(data_dir, f"fold{fold_idx}", 'train')
    test_path = os.path.join(data_dir, f"fold{fold_idx}", 'test')
    
    train_dataset = datasets.ImageFolder(root=train_path, transform=transform)
    val_dataset = datasets.ImageFolder(root=test_path, transform=transform)

    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

    optimizer = RMSprop(model.parameters(), lr=1e-5, momentum=0.9, weight_decay=5e-4)
    scheduler = StepLR(optimizer, step_size=5, gamma=0.1)
    
    best_loss = float('inf')
    best_model_path = os.path.join(data_dir, f"best_model_fold{fold_idx}.pth")

    for epoch in range(20):
        model.train()
        total_loss = 0

        with tqdm(train_loader, desc=f"Fold {fold_idx} Epoch {epoch + 1}/20", unit="batch") as t:
            for batch in t:
                inputs, labels = batch[0].to(device), batch[1].to(device)
                outputs = model(inputs, labels=labels)
                loss = outputs.loss

                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                total_loss += loss.item()
                t.set_postfix(loss=loss.item())

                del inputs, labels, outputs, loss
                torch.cuda.empty_cache()

        avg_loss = total_loss / len(train_loader)
        print(f"Fold {fold_idx} Epoch {epoch + 1}/20, Loss: {avg_loss}")

        scheduler.step()

        if avg_loss < best_loss:
            best_loss = avg_loss
            torch.save(model.state_dict(), best_model_path)
            print(f"New best model saved with loss: {best_loss}")



def evaluate_model(fold_idx):
    
    model_path = os.path.join(data_dir, f"best_model_fold{fold_idx}.pth")
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    model = Dinov2ForImageClassification.from_pretrained(
        "facebook/dinov2-small",
        num_labels=18,
        ignore_mismatched_sizes=True
    )

    model.load_state_dict(torch.load(model_path))
    model = model.to(device)
    model.eval()

    test_path = os.path.join(data_dir, f"fold{fold_idx}", 'test')
    val_dataset = datasets.ImageFolder(root=test_path, transform=transform)
    val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

    val_targets = []
    val_predictions = []

    with torch.no_grad():
        for inputs, targets in val_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            predictions = torch.argmax(outputs.logits, dim=1)
            val_targets.extend(targets.cpu().numpy())
            val_predictions.extend(predictions.cpu().numpy())

    cm = confusion_matrix(val_targets, val_predictions)
    print(f"Fold {fold_idx} Confusion Matrix:")
    print(cm)
    with open(os.path.join(data_dir, f'cm_{fold_idx}.txt'), 'w') as f:
        print(cm, file=f)

    report = classification_report(val_targets, val_predictions, output_dict=True, target_names=val_dataset.classes)
    print(report)
    with open(os.path.join(data_dir, f'report_{fold_idx}.txt'), 'w') as f:
        print(report, file=f)

    fold_results[fold_idx] = (report, cm)

In [8]:
for i in range(1, 6):
    train_model(i)
    
for i in range(1,6):
    evaluate_model(i)

Some weights of Dinov2ForImageClassification were not initialized from the model checkpoint at facebook/dinov2-small and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


cuda
Dinov2ForImageClassification(
  (dinov2): Dinov2Model(
    (embeddings): Dinov2Embeddings(
      (patch_embeddings): Dinov2PatchEmbeddings(
        (projection): Conv2d(3, 384, kernel_size=(14, 14), stride=(14, 14))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): Dinov2Encoder(
      (layer): ModuleList(
        (0-11): 12 x Dinov2Layer(
          (norm1): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
          (attention): Dinov2SdpaAttention(
            (attention): Dinov2SdpaSelfAttention(
              (query): Linear(in_features=384, out_features=384, bias=True)
              (key): Linear(in_features=384, out_features=384, bias=True)
              (value): Linear(in_features=384, out_features=384, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): Dinov2SelfOutput(
              (dense): Linear(in_features=384, out_features=384, bias=True)
              (dropout): Dropout(p=0.0, inplace=F

Fold 1 Epoch 1/20: 100%|██████████| 316/316 [02:05<00:00,  2.51batch/s, loss=1.76]


Fold 1 Epoch 1/20, Loss: 2.302917218283762
New best model saved with loss: 2.302917218283762


Fold 1 Epoch 2/20: 100%|██████████| 316/316 [02:06<00:00,  2.49batch/s, loss=2.23]


...

/home/gurol/anaconda3/envs/araproje/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/gurol/anaconda3/envs/araproje/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/gurol/anaconda3/envs/araproje/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capita

Fold 4 Confusion Matrix:
[[ 60   0  11   0   0   0  99   0   0  47   0   0  44  22   0  23   0   1]
 [  9   0   2   0   0   0  16   0   0  25   0   0  24   6   0   9   0   1]
 [  5   0  76   0   0   0  14   0   0  15   0   2   9   4   0   3  63   0]
 [  1   0   1   0   0   0   3   0   0   2   0   0   0   1   0   1   0   1]
 [  0   0   4   0   0   0   0   0   0   0   0   1   0   0   0   0  11   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0  21   0]
 [ 29   0   7   0   0   0 200   0   0   6   0   1  23   1   0   4   0   0]
 [  0   0   2   0   0   1   0   0   0   1   0   0   0   0   0   0  71   0]
 [  0   0   1   0   0   1   0   0  19   0   0   0   0   0   0   0   3   0]
 [ 25   0   8   0   0   0  21   0   0  95   0   0  29  31   0  16   0   0]
 [  7   0   1   0   0   0   9   0   0  16   0   0   3   3   0   1   0   0]
 [  2   0  23   0   0   0  13   0   0  11   0   1   8   1   0   4   8   1]
 [ 31   0   6   0   0   0  47   0   0  56   0   3  61  12   0  13   0   1]


/home/gurol/anaconda3/envs/araproje/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/gurol/anaconda3/envs/araproje/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/gurol/anaconda3/envs/araproje/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capita

Fold 5 Confusion Matrix:
[[ 53   0   9   0   0   0  99   0   0  52   0   0  53  30   0  10   1   0]
 [ 13   0   4   0   0   0  16   0   0  30   0   0  15   9   0   4   0   0]
 [  3   0  72   0   0   0  12   0   1  14   0   0  16   2   0   6  65   0]
 [  0   0   1   0   0   0   5   0   0   2   0   0   1   1   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0  16   0]
 [  0   0   0   0   0   0   0   0   1   0   0   0   0   0   0   0  20   0]
 [ 31   0   7   0   0   0 191   0   0   1   0   1  31   4   0   4   1   0]
 [  0   0   3   0   0   0   0   0   0   0   0   0   0   0   0   0  73   0]
 [  0   0   0   0   0   0   0   0  20   0   0   0   0   0   0   0   3   0]
 [ 27   0   9   0   0   0  17   0   0 101   0   0  33  25   0  12   0   0]
 [  8   0   1   0   0   0   6   0   0  11   0   0   2  11   0   1   0   0]
 [  2   0  21   0   0   0  14   0   0  14   0   1  11   1   0   0   8   0]
 [ 22   0  12   0   0   0  56   0   0  54   0   1  52  22   0  11   0   0]


/home/gurol/anaconda3/envs/araproje/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/gurol/anaconda3/envs/araproje/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/gurol/anaconda3/envs/araproje/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capita

In [9]:
print(fold_results)

{1: ({'AmorfHead': {'precision': 0.16943521594684385, 'recall': 0.16612377850162866, 'f1-score': 0.16776315789473684, 'support': 307.0}, 'AsymmetricNeck': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 91.0}, 'CurlyTail': {'precision': 0.3787878787878788, 'recall': 0.39267015706806285, 'f1-score': 0.3856041131105398, 'support': 191.0}, 'DoubleHead': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 10.0}, 'DoubleTail': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 16.0}, 'LongTail': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 21.0}, 'NarrowAcrosome': {'precision': 0.3953488372093023, 'recall': 0.6900369003690037, 'f1-score': 0.5026881720430108, 'support': 271.0}, 'Normal': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 76.0}, 'PinHead': {'precision': 0.8636363636363636, 'recall': 0.8636363636363636, 'f1-score': 0.8636363636363636, 'support': 22.0}, 'PyriformHead': {'precision': 0.2358490566037736, 'recall': 0.4

In [10]:
with open(os.path.join(data_dir, 'convnextv2tiny_report.txt'), 'w') as f:
    print(fold_results, file=f)

In [11]:
def save_confusion_matrix(cm, classes, output_path, fontsize=11):
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap="OrRd", cbar=True, 
                xticklabels=classes, yticklabels=classes, annot_kws={"size": fontsize})
    
    plt.xlabel("Predicted", fontsize=fontsize)
    plt.ylabel("Actual", fontsize=fontsize)
    plt.title("Confusion Matrix", fontsize=fontsize)
    plt.xticks(rotation=90, fontsize=fontsize)
    plt.yticks(rotation=0, fontsize=fontsize)
    
    plt.tight_layout() 
    plt.savefig(output_path)
    plt.close()
    print(f"Confusion matrix saved to {output_path}")

classes = datasets.ImageFolder(root=os.path.join(data_dir, 'fold1', 'test'), transform=transform).classes
for i in range(1, 6):
    save_confusion_matrix(fold_results[i][1], classes, os.path.join(data_dir, f'convnextv2tiny_fold{i}.png'))

Confusion matrix saved to datasets/Boya3/convnextv2tiny_fold1.png
Confusion matrix saved to datasets/Boya3/convnextv2tiny_fold2.png
Confusion matrix saved to datasets/Boya3/convnextv2tiny_fold3.png
Confusion matrix saved to datasets/Boya3/convnextv2tiny_fold4.png
Confusion matrix saved to datasets/Boya3/convnextv2tiny_fold5.png


In [12]:
def matrix_sum(fold_results):
    cumulative_cm = None


    for fold_idx, (report, cm) in fold_results.items():
        # Combine confusion matrices
        if cumulative_cm is None:
            cumulative_cm = cm
        else:
            cumulative_cm += cm

    return cumulative_cm

cumulative_cm = matrix_sum(fold_results)

print("Cumulative Confusion Matrix:")
print(cumulative_cm)
with open(os.path.join(data_dir, 'cumulative_cm.txt'), 'w') as f:
    print(cumulative_cm, file=f)
save_confusion_matrix(cumulative_cm, classes, os.path.join(data_dir, 'cumulative_cm.png'))

Cumulative Confusion Matrix:
[[ 276    0   47    0    0    0  514    0    0  272    0    0  233  114
     0   78    1    2]
 [  68    0   20    0    0    0   83    0    0  127    0    0   86   42
     0   30    1    1]
 [  18    0  361    0    0    0   79    0    1   81    0    2   69   11
     0   37  296    0]
 [   5    0    3    0    0    0   21    0    0    8    0    0    4    4
     0    5    1    1]
 [   0    0   10    0    0    0    0    0    0    0    0    1    1    0
     0    0   69    0]
 [   0    0    0    0    0    0    0    0    2    0    0    0    0    0
     0    0  104    0]
 [ 200    0   36    0    0    0  943    0    0   17    0    2  129    8
     0   18    2    0]
 [   0    0   14    0    0    1    1    0    4    1    0    0    0    0
     0    0  359    0]
 [   0    0    3    0    0    1    0    0  100    0    0    0    0    0
     0    0   13    0]
 [ 123    0   41    0    0    0   76    0    1  544    1    1  161  113
     0   60    2    0]
 [  30    0    4    0

In [13]:
def classification_report_from_confusion_matrix(confusion_matrix, class_names):
    y_true = np.zeros(np.sum(confusion_matrix), dtype=int)
    y_pred = np.copy(y_true)

    i = 0
    for target in range(len(confusion_matrix)):
        for pred in range(len(confusion_matrix)):
            n = confusion_matrix[target][pred]
            y_true[i:i+n] = target
            y_pred[i:i+n] = pred
            i += n

    return classification_report(y_true, y_pred, digits=4, target_names=class_names)

avg_report = classification_report_from_confusion_matrix(cumulative_cm, classes)
print (avg_report)
with open(os.path.join(data_dir, 'average_report.txt'), 'w') as f:
    print(avg_report, file=f)

                precision    recall  f1-score   support

     AmorfHead     0.2125    0.1796    0.1946      1537
AsymmetricNeck     0.0000    0.0000    0.0000       458
     CurlyTail     0.3546    0.3780    0.3659       955
    DoubleHead     0.0000    0.0000    0.0000        52
    DoubleTail     0.0000    0.0000    0.0000        81
      LongTail     0.0000    0.0000    0.0000       106
NarrowAcrosome     0.4020    0.6959    0.5096      1355
        Normal     0.0000    0.0000    0.0000       380
       PinHead     0.8333    0.8547    0.8439       117
  PyriformHead     0.2348    0.4844    0.3163      1123
     RoundHead     0.0000    0.0000    0.0000       201
     ShortTail     0.1250    0.0056    0.0107       359
   TaperedHead     0.2126    0.2583    0.2332      1150
     ThickNeck     0.2175    0.1520    0.1790       934
      ThinNeck     0.0000    0.0000    0.0000       180
   TwistedNeck     0.2273    0.1404    0.1735       855
   TwistedTail     0.6930    0.8893    0.7790  

/home/gurol/anaconda3/envs/araproje/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/gurol/anaconda3/envs/araproje/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/gurol/anaconda3/envs/araproje/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capita